In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import broadcast

In [ ]:
spark = SparkSession.builder.appName("OptimisationNotes").getOrCreate()

In [ ]:
dataframe_logs = spark.read.parquet("../lab_2/data/output/logs_hourly/logs_hourly.parquet")

In [ ]:
dataframe_logs.explain(True)

In [ ]:
dataframe_opt = dataframe_logs.repartition(8)
dataframe_opt.cache()

In [ ]:
dataframe_users = spark.read.csv("../lab_1/data/users.csv", header=True, inferSchema=True)
dataframe_joined = dataframe_opt.join(broadcast(dataframe_users), dataframe_opt.user_id == dataframe_users.id)

## Tests de Partitionnement

Test de différents niveaux de partitionnement (2, 4, 8) et mesure des performances.

In [ ]:
import time

def test_partitioning(param_dataframe, param_num_partitions):
    print(f"Test avec {param_num_partitions} partitions")

    start_time = time.time()

    # Repartitionner et supprimer le cache pour avoir des mesures précises
    df_repartitioned = param_dataframe.repartition(param_num_partitions)
    df_repartitioned.unpersist()

    count = df_repartitioned.count()

    execution_time = time.time() - start_time

    enregistrements_par_partition = count // param_num_partitions

    print(f"Nombre d'enregistrements       : {count:,}")
    print(f"Nombre de partitions           : {df_repartitioned.rdd.getNumPartitions()}")
    print(f"Enregistrements par partition  : ~{enregistrements_par_partition:,}")
    print(f"Temps d'exécution              : {execution_time:.2f} secondes")
    print(f"{'='*60}")

    return df_repartitioned, execution_time

In [ ]:
# Test avec 2 partitions
dataframe_2_partitions, time_2 = test_partitioning(dataframe_logs, 2)

In [ ]:
# Test avec 4 partitions
dataframe_4_partitions, time_4 = test_partitioning(dataframe_logs, 4)

In [ ]:
# Test avec 8 partitions
dataframe_8_partitions, time_8 = test_partitioning(dataframe_logs, 8)

In [ ]:
print("\n" + "="*60)
print("RÉSUMÉ DES PERFORMANCES")
print("="*60)
print(f"2 partitions : {time_2:.2f}s")
print(f"4 partitions : {time_4:.2f}s")
print(f"8 partitions : {time_8:.2f}s")
print("="*60)

times = {2: time_2, 4: time_4, 8: time_8}
best_partitions = min(times, key=times.get)
print(f"\nMeilleure configuration : {best_partitions} partitions ({times[best_partitions]:.2f}s)")
print("="*60)

## Checkpoint pour Tolérance aux Pannes

In [ ]:
import os

checkpoint_dir = "./checkpoint"
os.makedirs(checkpoint_dir, exist_ok=True)

# Configurer le checkpoint dans Spark
spark.sparkContext.setCheckpointDir(checkpoint_dir)

print(f"Répertoire de checkpoint configuré : {checkpoint_dir}")

In [ ]:
# Appliquer le checkpoint sur le DataFrame optimisé avec 8 partitions
dataframe_checkpointed = dataframe_8_partitions.checkpoint()

print("Checkpoint appliqué sur le DataFrame")
print(f"Nombre de partitions : {dataframe_checkpointed.rdd.getNumPartitions()}")
print(f"Nombre d'enregistrements : {dataframe_checkpointed.count():,}")

In [ ]:
spark.stop()